<a href="https://colab.research.google.com/github/kaho1156/Clinical-Data-Engineering-Privacy-Pipeline/blob/main/Clinical_data_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import random
import hashlib
import sqlite3
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker with Australian locale settings
fake = Faker('en_AU')

# ==========================================
# 1. DATA GENERATION (Simulating Raw Clinical Data)
# ==========================================
def generate_clinical_data(n=2000):
    """
    Generates a high-fidelity synthetic clinical dataset
    simulating a hospital EMR (Electronic Medical Record) environment.
    """
    data = []
    for _ in range(n):
        dob = fake.date_of_birth(minimum_age=0, maximum_age=95)
        # Randomize admission within the last 30 days
        adm_dt = datetime.now() - timedelta(days=np.random.randint(1, 30))

        data.append({
            "Name": fake.name(),
            "Patient ID": f"PID-{np.random.randint(100000, 999999)}",
            "Sex at Birth": np.random.choice(["M", "F"]),
            "Gender Identity": np.random.choice(["Male", "Female", "Non-binary", "Other"]),
            "Date of Birth": dob.strftime("%Y-%m-%d"),
            "Age": datetime.now().year - dob.year,
            "Bed ID": f"B-{np.random.randint(1, 20)}",
            "Length of Stay": np.random.randint(1, 100),
            "Clinician": f"Dr. {fake.last_name()}",
            "Specialty": np.random.choice(["Cardiology", "Neurology", "ICU", "General Med"]),
            "Medicare Card Number": fake.bothify(text='#### ##### #'),
            # Conditional Logic: Private policy generated only for some patients
            "Policy Number": random.choice([fake.bothify(text='POL-#########'), ""]),
            "Tel Ext": np.random.randint(1000, 9999),
            "Admission Source": np.random.choice(["Emergency", "GP Referral", "Transfer"]),
            "GP (D)": f"Dr. {fake.last_name()}",
            "Hosp. Admission": adm_dt.strftime("%Y-%m-%d %H:%M"),
            "Create UserID (D)": f"USER-{np.random.randint(10, 99)}"
        })
    return pd.DataFrame(data)

# ==========================================
# 2. DE-IDENTIFICATION PIPELINE (Data Privacy Compliance)
# ==========================================
def deidentify_healthcare_data(df):
    """
    Applies Masking, Hashing, and Generalization to protect PII
    in compliance with the Privacy Act 1988 (Australia).
    """
    processed = df.copy()

    # [MASKING] - Redacting highly sensitive personal identifiers
    processed['Name'] = "[REDACTED]"
    processed['Medicare Card Number'] = processed['Medicare Card Number'].apply(lambda x: f"XXXX-XXXX-{x[-1]}")
    processed['Policy Number'] = processed['Policy Number'].apply(lambda x: "CONFIDENTIAL" if x != "" else "N/A")
    processed['Tel Ext'] = "XXXX"

    # [HASHING] - Pseudonymization to maintain relational integrity for longitudinal analysis
    processed['Patient ID'] = processed['Patient ID'].apply(
        lambda x: hashlib.sha256(x.encode()).hexdigest()[:12].upper()
    )

    # [GENERALIZATION] - Reducing precision to mitigate re-identification risk
    # 1. Transform Date of Birth to Birth Year
    processed['Birth Year'] = pd.to_datetime(processed['Date of Birth']).dt.year
    processed.drop(columns=['Date of Birth'], inplace=True)
    # 2. Categorize Age into Age Groups (e.g., 70s, 80s)
    processed['Age Group'] = (processed['Age'] // 10 * 10).astype(str) + "s"
    processed.drop(columns=['Age'], inplace=True)

    return processed

# ==========================================
# 3. ETL & KPI ANALYSIS (Clinical Operations Monitoring)
# ==========================================
def process_hospital_data(df):
    """Phase 3: ETL - Ensuring data formats are optimized for SQL operations."""
    processed = df.copy()
    processed['Hosp_Admission'] = pd.to_datetime(processed['Hosp. Admission'])
    return processed

def run_clinical_kpis(conn):
    """Phase 4: Analytics - Monitoring core hospital performance indicators."""
    scenarios = {
        # KPI 1: Patient-Centric Care (Identify upcoming birthdays)
        "Upcoming Birthdays (Next 3 Days)": """
            SELECT Name, [Patient ID], [Specialty],
            CASE
                WHEN strftime('%m-%d', [Date of Birth]) BETWEEN
                     strftime('%m-%d', 'now') AND strftime('%m-%d', 'now', '+3 days')
                THEN 'Y' ELSE 'N'
            END AS Is_Birthday_Soon
            FROM clinical_data
            WHERE Is_Birthday_Soon = 'Y';
        """,

        # KPI 2: High-Turnover Monitoring (Admitted within last 72 hours)
        "New Patients (< 3 Days)": """
            SELECT Name, [Patient ID], [Hosp. Admission], Specialty
            FROM clinical_data
            WHERE julianday('now') - julianday([Hosp. Admission]) <= 3;
        """,

        # KPI 3: Revenue Cycle Management (Private health fund auditing)
        "Private Insurance Status": """
            SELECT COUNT(*) AS Patient_Count,
            CASE WHEN [Policy Number] = "" THEN "N" ELSE "Y" END AS HAS_INSURANCE
            FROM clinical_data GROUP BY HAS_INSURANCE;
        """,

        # KPI 4: Patient Flow & LOS (Monitor stays exceeding 14 days)
        "Long Stay Alert (> 14 Days)": """
            SELECT Name, [Patient ID], [Length of Stay], [Specialty]
            FROM clinical_data
            WHERE [Length of Stay] > 14;
        """
    }

    for title, query in scenarios.items():
        print(f"\n📊 KPI Alert: {title}")
        display(pd.read_sql(query, conn))

# --- Main System Execution ---
raw_df = generate_clinical_data(2000)
with sqlite3.connect(":memory:") as conn:
    df_processed = process_hospital_data(raw_df)
    df_processed.to_sql("clinical_data", conn, index=False)
    run_clinical_kpis(conn)


📊 KPI Alert: Upcoming Birthdays (Next 3 Days)


,Name,Patient ID,Specialty,Is_Birthday_Soon
0,Michael Craig,PID-397924,Neurology,Y
1,Jeremy Davis,PID-722541,General Med,Y
2,John Smith,PID-931540,ICU,Y
3,Renee King,PID-679097,Neurology,Y
4,Robin Shannon,PID-973401,General Med,Y
5,Crystal Shea,PID-871842,General Med,Y
6,Jennifer Price,PID-192186,ICU,Y
7,Daniel Parks,PID-532969,Neurology,Y
8,Patricia Stevenson,PID-405799,Cardiology,Y
9,Steven Strickland,PID-148804,Neurology,Y



📊 KPI Alert: New Patients (< 3 Days)


,Name,Patient ID,Hosp. Admission,Specialty
0,Jacqueline Stevens,PID-818944,2026-01-08 00:33,Cardiology
1,Samantha Price,PID-861243,2026-01-07 00:33,Cardiology
2,Colleen Williams,PID-991096,2026-01-07 00:33,Neurology
3,Victoria Herman,PID-982966,2026-01-07 00:33,Cardiology
4,Melissa Brennan,PID-226294,2026-01-07 00:33,ICU
...,...,...,...,...
105,Dawn Mcdonald,PID-570641,2026-01-07 00:33,Cardiology
106,Cynthia Avery,PID-262621,2026-01-08 00:33,Cardiology
107,Vanessa Harrison,PID-840566,2026-01-07 00:33,Neurology
108,Lisa Sharp,PID-582788,2026-01-08 00:33,Neurology



📊 KPI Alert: Private Insurance Status


,Patient_Count,HAS_INSURANCE
0,1023,N
1,977,Y



📊 KPI Alert: Long Stay Alert (> 14 Days)


,Name,Patient ID,Length of Stay,Specialty
0,Joseph Gonzales,PID-261126,85,ICU
1,Michael Craig,PID-397924,72,Neurology
2,Roy Smith,PID-621801,38,Cardiology
3,Karen Bird,PID-343399,21,General Med
4,Grace Griffin,PID-431248,51,General Med
...,...,...,...,...
1717,Lisa Mccullough,PID-632645,19,Neurology
1718,Troy Case,PID-552326,91,Cardiology
1719,Carlos Johnson,PID-106107,36,General Med
1720,Ronnie Osborn,PID-502572,55,ICU
